In [ ]:
import os
import azureml.core
import pandas as pd
import numpy as np
import logging
import warnings

from pandas.tseries.frequencies import to_offset

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

np.set_printoptions(precision=4, suppress=True, linewidth=120)

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.featurization import FeaturizationConfig
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from azureml.train.estimator import Estimator

from pandas.io.json import json_normalize

import datetime
from dateutil.tz import tzutc
#!pip install jupyterlab_code_formatter
#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/forecasting-orange-juice-sales/auto-ml-forecasting-orange-juice-sales.ipynb

In [ ]:
print("This notebook was created using version 1.18.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

In [ ]:
%run ./helper.ipynb

In [ ]:
#read input files
time_column_name = 'End of Month'
time_column_name1 = 'Snapshot Date Short'
talenthist = pd.read_csv("TalentHistory.csv", parse_dates=[time_column_name])
pipehist = pd.read_csv("PipelineHistory.csv", parse_dates=[time_column_name,time_column_name1])
revhist = pd.read_csv("RevenueHistory.csv", parse_dates=[time_column_name])
opphist = pd.read_csv("OppHistory.csv", parse_dates=[time_column_name])
projhist = pd.read_csv("ProjectHistory.csv", parse_dates=[time_column_name])
time_column_name2 = 'Latest Snapshot Date Short'
pipetrend = pd.read_csv("PipelineTrend.csv", parse_dates=[time_column_name2])
display(talenthist.tail())
display(pipehist.tail())
display(revhist.tail())
display(opphist.tail())
display(projhist.tail())
display(pipetrend.tail())

In [ ]:
#download SP500 closing prices
#if you get an error after executing the code, try adding below. 
#!pip install pandas-datareader

pd.set_option('display.float_format', lambda x: '%.6f' % x)

start = datetime.datetime(2016, 1, 1)
end = today
#pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
#SP500 = web.DataReader(['sp500'], 'fred', start, end)
#display(SP500.head())

import pandas_datareader as pdr

SP500 = pdr.get_data_fred('sp500', start, end)
SP500 = SP500.reset_index(level=SP500.index.names)
SP500['End_of_Month'] = SP500['DATE']+ pd.offsets.MonthEnd(0)
SP500 = SP500.sort_values('DATE').groupby('End_of_Month').last()
#SP500['Monthly_Return'] = (SP500['sp500']/ SP500['sp500'].shift(1)) -1
#SP500['sp500_1M_Previous'] = SP500['sp500'].shift(1)
SP500 = SP500.drop('DATE', axis=1)
SP500 = SP500.reset_index(level=SP500.index.names)

#Drop all Not a number values using drop method.
SP500.dropna(inplace = True)
print("\n SP500: \n")
display(get_df_name(SP500))
print("\n DF Info: \n")
display(SP500.info(verbose=True))
print("\n SP500 Describe: \n")
display(SP500.describe(include='all').transpose().head())
print("\n SP500 Head: \n")
display(SP500.head())
print("\n SP500 Tail: \n")
display(SP500.tail())
SP500['sp500'].plot(title='S&P 500')

In [ ]:
#create an Experiment. An Experiment corresponds to a prediction problem you are trying to solve, while a Run corresponds to a specific approach to the problem.
ws = Workspace.from_config()

# choose a name for the run history container in the workspace
experiment_name = 'revenue-time-forecast1'

experiment = Experiment(ws, experiment_name)

output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-bi-ml-devqa01"

compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
print('Found existing cluster, use it.')

In [ ]:
#Rename Studio, filter studio and prep data
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#exclude_studio = ["Regional Bay Area", "Regional ChiCamNY", "Palo Alto", "Singapore", "Unknown", "North America"]

#display(exclude_studio)
talenthist1 = talenthist.copy()
pipehist1 = pipehist.copy()
opphist1 = opphist.copy()
projhist1 = projhist.copy()
revhist1 = revhist.copy()
pipetrend1 = pipetrend.copy()
talenthist1.columns = talenthist1.columns.astype(str).str.replace("Contrib ", "")
pipehist1.columns = pipehist1.columns.astype(str).str.replace("Contrib ", "")
pipetrend1.columns = pipetrend1.columns.astype(str).str.replace("Contrib ", "")
opphist1.columns = opphist1.columns.astype(str).str.replace("Managing ", "")
projhist1.columns = projhist1.columns.astype(str).str.replace("Managing ", "")
revhist1.columns = revhist1.columns.astype(str).str.replace("Managing ", "")
#talenthist1 = talenthist.rename(columns={"Contrib Studio Name": "Studio"})
#pipehist1 = pipehist.rename(columns={"Contrib Studio Name": "Studio"})

#revhist1 = revhist.rename(columns={"Managing Studio Name": "Studio"})
pipetrend1 = pipetrend1.rename(columns={"Latest Snapshot Date Short": "End of Month",
                                       #"Contrib Studio Name": "Studio", 
                                       "Relative Snapshot Month Offset": "Relative Month Offset"})
#opphist1 = opphist.rename(columns={"Managing Studio Name": "Studio"})
#projhist1 = projhist.rename(columns={"Managing Studio Name": "Studio"})
#revhist1 = revhist1.query('End_of_Month < @current_eom').reset_index(drop=True)
display(pipetrend1)
cols_float1 = opphist1.filter(like='Rate', axis=1).columns
remove_percetage(opphist1, cols_float1)
cols_float2 = pipehist1.filter(like='Yield', axis=1).columns
remove_percetage(pipehist1, cols_float2)

talenthist1 = data_prep(talenthist1)
pipehist1 = data_prep(pipehist1)
revhist1 = data_prep(revhist1)
pipetrend1 = data_prep(pipetrend1)
opphist1 = data_prep(opphist1)
projhist1 = data_prep(projhist1)
display(pipetrend1)
coerce_df_columns_to_numeric(talenthist1)
coerce_df_columns_to_numeric(revhist1)
coerce_df_columns_to_numeric(opphist1)
coerce_df_columns_to_numeric(projhist1)
coerce_df_columns_to_numeric(pipehist1)
coerce_df_columns_to_numeric(pipetrend1)
display(pipetrend1)
show_stats(pipehist1)

#revhist1 = revhist1.query('Fin_Entity_ID not in ["SGP",0]').copy()
pipehist1['Snapshot_Date_Short'] = pipehist1['Snapshot_Date_Short'] + pd.offsets.MonthEnd(0) #make sure all End of Month are end of month dates
pipehist1['End_of_Month'] = pipehist1['End_of_Month'] + pd.offsets.MonthEnd(0) #make sure all End of Month are end of month dates
pipetrend1['End_of_Month'] = pipetrend1['End_of_Month'] + pd.offsets.MonthEnd(0) #make sure all End of Month are end of month dates
opphist1['End_of_Month'] = opphist1['End_of_Month'] + pd.offsets.MonthEnd(0) 
projhist1['End_of_Month'] = projhist1['End_of_Month'] + pd.offsets.MonthEnd(0) 

#Sort
pipehist1.sort_values(by=['Snapshot_Date_Short','End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
revhist1.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
talenthist1.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
pipetrend1.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
opphist1.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
projhist1.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)

display(talenthist1.tail())
display(pipehist1.tail())
display(revhist1.tail())
display(opphist1.tail())
display(projhist.tail())
display(pipetrend1.tail())

In [ ]:
pipetrend1

In [ ]:
revhist1

In [ ]:
display(pipehist1.query('Relative_Snapshot_Month_Offset == 0 and Fin_Entity_ID == "USA"'))

In [ ]:
print("\n revhist1: \n")
#revhist1.to_csv("revhist1.csv", index=False)
#show_stats(revhist1)
#display(revhist1.query('Fin_Entity_ID==@entity_debug').reset_index(drop=True))
#pivot
revhist_pivot = revhist1.pivot_table(index='End_of_Month',#columns = ['Fin_Entity_ID'], 
                                     values='Revenue', 
                                     #aggfunc=['sum','count'], 
                                     margins=False)
#revhist_pivot = revhist_pivot.rename(columns={'sum':'Revenue'})
revhist_pivot = revhist_pivot.reset_index(level=revhist_pivot.index.names)
#revhist_pivot['End_of_Month'] = pd.to_datetime(revhist_pivot['End_of_Month'])#.dt.date
#revhist_pivot.index.name = 'Date'
revhist_pivot.fillna(0, inplace=True)
revhist_pivot.tail(10)

In [ ]:
# Cross join to get all unquie values of End_of_Month and Studio
d = pd.date_range(start_date,end_date,freq='m')
eom = pd.DataFrame(d, columns=['End_of_Month'])
#eom = pipehist_pivot1[['End_of_Month']].drop_duplicates(subset='End_of_Month',keep="last").sort_values(by=['End_of_Month']).reset_index(drop=True)
studio = pd.DataFrame(talenthist1['Fin_Entity_ID'].unique(), columns=['Fin_Entity_ID'])
eom['key'] = 0
studio['key'] = 0
eom_studio = pd.merge(eom, studio, on=['key'])
eom_studio = eom_studio.replace(np.nan, "Unknown", regex=True)
eom_studio.drop(columns=['key'], axis=1, inplace = True)
print("\n eom_studio: \n")
display(eom_studio.info(verbose=True))
eom_studio

In [ ]:
#Merge eom_studio with Revenue, Talent, Opportunity, Project
from functools import reduce
nan_value = 0
merge_dfs = [eom_studio, revhist1, talenthist1, opphist1, projhist1, pipetrend1]

merge_df = reduce(lambda left,right: pd.merge(left, right, 
                                              how='left',
                                              on=['End_of_Month','Fin_Entity_ID']), 
                  merge_dfs).fillna(nan_value)
coerce_df_columns_to_numeric(merge_df)
merge_df = merge_df.replace(np.nan, 0, regex=True)
#merge_df = merge_df.query('Fin_Entity_ID!="SGP"') # Exclude Singapore rows
merge_df['Relative_Month_Offset'] = round((merge_df['End_of_Month'] - current_eom)/np.timedelta64(1,'M'),0).astype(int)
merge_df['Total_Headcount'] = merge_df['Headcount'] + merge_df['Headcount_Contingent']
merge_df.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
#merge_df.drop(columns=['Current_Opp._Period_Value','Conversions'], axis=1, inplace = True)
print("\n merge_df: \n")
merge_df.to_csv("merge_df.csv", index=False)
#show_stats(merge_df)
merge_df.query('Fin_Entity_ID==@entity_debug')

In [ ]:
#Merge with SP500
eom1 = pd.DataFrame(d, columns=['End_of_Month'])
sp500_df = pd.merge(eom1, SP500, how='left', on=['End_of_Month'])
#sp500_df = sp500_df.ffill(axis = 0)
sp500_df = sp500_df.sort_values(by=['End_of_Month']).reset_index(drop=True)
display(sp500_df)
merge_df1 = pd.merge(merge_df, sp500_df, how='left', on=['End_of_Month'])
coerce_df_columns_to_numeric(merge_df1)
merge_df1.drop(columns=['Headcount','Headcount_Contingent'], axis=1, inplace = True) #,'Total_Headcount','Monthly_Return','sp500_1M_Previous'
merge_df1 = merge_df1.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
merge_df1.to_csv("merge_df1.csv", index=False)
show_stats(merge_df1)

In [ ]:
#Calculate pipeline month offset for corresponding snapshot dates
n_test_periods = 4
rolling_pipeline_period = n_test_periods
pipehist_long = pipehist1.copy()
pipehist_long["Relative_Offset"] = (pipehist_long['Relative_Month_Offset'] - pipehist_long['Relative_Snapshot_Month_Offset'])
pipehist_long = pipehist_long \
                .query('Relative_Offset >= -@rolling_pipeline_period and ' + 
                        'Relative_Offset < @rolling_pipeline_period and ' +
                        'Snapshot_Date_Short <= @end_date') 
cols='Relative_Offset'
pipehist_long[cols] = pipehist_long[cols].apply(pd.to_numeric, errors='coerce')
#pipehist2 = pipehist2.query('Relative_Offset>=-11 and Relative_Offset<=0')
#pipehist_long["Relative_Offset"] = pipehist_long["Relative_Offset"].astype(str).str.zfill(2) + 'M Pipeline'
#pipehist_long["Relative_Offset"] = pipehist_long["Relative_Offset"].str.replace('0','+')
#pipehist_long["Relative_Offset"] = pipehist_long["Relative_Offset"].str.replace('++M Pipeline','+0M Pipeline')
#pipehist_long["Relative_Offset"] = "Relative" + pipehist_long["Relative_Offset"].str.replace("0-","-0")
pipehist_long.sort_values(by=['Snapshot_Date_Short','End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
print("\n pipehist_long: \n")
pipehist_long.to_csv("pipehist_long.csv", index=False)
display(pipehist_long.query('Fin_Entity_ID==@entity_debug'))
pipehist_long.info()

In [ ]:
#Convert pipe history from long to wide to match month end grain

pipehist_wide =  pipehist_long \
                .pivot_table(index=['End_of_Month','Relative_Month_Offset','Fin_Entity_ID'], 
                                    columns='Relative_Offset', 
                                    values=['Pipeline','Pipeline_3_Month_Rolling_Avg'], 
                                    aggfunc=np.mean,
                                    margins=False)
pipehist_wide = pipehist_wide.reset_index(level=pipehist_wide.index.names)#.reset_index(drop=True)
cols = list(pipehist_wide.select_dtypes(include='float64').columns)
coerce_df_columns_to_numeric(pipehist_wide)
pipehist_wide['End_of_Month'] = pd.to_datetime(pipehist_wide['End_of_Month'])
#pipehist_wide.rename(columns={0:'Pipeline_3_Month_Rolling_Avg'}, inplace=True)
pipehist_wide = pipehist_wide.sort_values(by=['End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
print("\n pipehist_wide: \n")
display(pipehist_wide.info())
#pipehist_wide.query('Fin_Entity_ID==@entity_debug')
pipehist_wide

In [ ]:
# Merge merge_df with Pipeline Long History 
#pipe_offset = ['Relative-00','Relative-01','Relative-02','Relative-03',
#               'Relative-04','Relative-05','Relative-06'
#               ,'Relative-07', 'Relative-08','Relative-09','Relative-10','Relative-11'
#              ]
#1merge_final = pd.merge(merge_df1, pipehist_long.query('Relative_Offset == 0'), how='left', on=['End_of_Month','Relative_Month_Offset','Fin_Entity_ID'])
#1merge_final.drop(columns=['Snapshot_Date_Short','Relative_Snapshot_Month_Offset'], axis=1, inplace = True) 
#New Code
pipehist_long1 = pipehist_long.copy()#.query('Relative_Offset >= 0 and Relative_Offset < @n_test_periods').copy()
merge_final = pd.merge(merge_df1, pipehist_long1, how='left', on=['End_of_Month','Relative_Month_Offset','Fin_Entity_ID'])
#merge_final.drop(columns=['Snapshot_Date_Short','Relative_Snapshot_Month_Offset'], axis=1, inplace = True) 
#New Code end
#merge_final['End_of_Month'] = pd.to_datetime(merge_final['End_of_Month'])
#revpipe_final['Relative_Month_Offset'] = round((revpipe_final['End_of_Month'] - current_eom)/np.timedelta64(1,'M'),0)
coerce_df_columns_to_numeric(merge_final)

# Filter for only next 11 months rows w.r.t. to snapshot date
#revpipe_final1 = revpipe_final.query('Relative_Offset in @pipe_offset').reset_index(drop=True)
#merge_final = merge_final.replace(np.nan, 0, regex=True)
merge_final = merge_final.sort_values(by=['Snapshot_Date_Short','End_of_Month','Fin_Entity_ID']).reset_index(drop=True)
display(merge_final.info())
merge_final.rename(columns = {-12: '-12M Pipeline', -11: '-11M Pipeline', -10: '-10M Pipeline', -9: '-9M Pipeline', -8: '-8M Pipeline', -7: '-7M Pipeline',\
                              -6: '-6M Pipeline', -5: '-5M Pipeline', -4: '-4M Pipeline', -3: '-3M Pipeline', -2: '-2M Pipeline', -1: '-1M Pipeline',\
                             0: '0M Pipeline', 1: '1M Pipeline', 2: '2M Pipeline', 3: '3M Pipeline', 4: '4M Pipeline', 5: '5M Pipeline',\
                             6: '6M Pipeline', 7: '7M Pipeline', 8: '8M Pipeline', 9: '9M Pipeline', 10: '10M Pipeline', 11: '11M Pipeline'}, inplace = True)
print("\n merge_final: \n")
merge_final.to_csv("merge_final.csv", index=False)
show_stats(merge_final)
merge_final.query('Fin_Entity_ID==@entity_debug')

In [ ]:
#To distinguish the individual time-series, we define the time_series_id_column_names - the columns whose values determine the boundaries between time-series:
target_column_name = 'Revenue'
time_column_name = 'End_of_Month'
time_series_id_column_names = ['Relative_Offset','Fin_Entity_ID']
sort_list = [time_column_name] + list(set(time_series_id_column_names))
df = merge_final#merge_df1

In [ ]:
#Data Split
#split the data into a training and a testing set for later forecast evaluation. The test set will contain the final test_size months of observed sales for each time-series. 
#The splits should be stratified by series, so we use a group-by statement on the time series identifier columns.
#test_size = .11 #in percentage
#n_test_periods =  int(tseries * test_size)
#n_test_periods = rolling_pipeline_period

In [ ]:
# Merge Fin Entity ID to do the revenue forecast at IDEO level first
#Filter final dataframe for current month + forecast horizon of n_test_periods
#merge_final1 = merge_final.copy()#['End_of_Month'] + pd.offsets.MonthEnd(n_test_periods) 
end_date1 = today + pd.offsets.MonthEnd(n_test_periods)
print(n_test_periods, end_date1, last_eom)
df1=df.copy()
#df1 = df.query('Relative_Month_Offset < @n_test_periods').sort_values(by=time_column_name).reset_index(drop=True)
df1.info()
display(df1.query('Fin_Entity_ID==@entity_debug').sort_values(by=time_column_name))
#display(df1)

In [ ]:
tseries = df1.groupby(time_column_name).ngroups
display('Data contains {0} individual time_column_name.'.format(tseries))
test_size = round(n_test_periods/tseries,2)
display('Test Size: ',test_size, 'Test Periods: ', n_test_periods)
display(df1[time_column_name].unique())

In [ ]:
#df = merge_final.drop(columns=['End_of_Month','Relative_Month_Offset']).sort_values(by=sort_list)# axis=1, inplace = True)
nseries = df1.groupby(time_series_id_column_names).ngroups
display('Data contains {0} individual time-series.'.format(nseries))
display(sort_list)
display(df1.groupby(time_series_id_column_names).count())

In [ ]:
#train, test = split_full_for_forecasting(df=df1, time_column_name=time_column_name, grain_column_names=time_series_id_column_names, test_split=test_size)
train, test = split_last_n_by_series_id(df1, n_test_periods)
#train = df1.query('Relative_Snapshot_Month_Offset != 0')
#test = df1.drop(train.index)
#test = merge_final1.query('(End_of_Month >= @current_eom)')#.sort_values(by=time_column_name)#.reset_index(drop=True)
#train = merge_final1.drop(test.index)#.sort_values(by=sort_list)#('(
display(train)
display(test)
#df1 = df.copy()
#train = df1.sample(frac=train_size, random_state=0)
#test = df1.drop(train.index)
#train_size = int(df.shape[0] * 0.9)
#train = df[:n_test_periods]
#test = df[n_test_periods:]
train.to_csv(r'./revtime_train.csv', index = None, header=True)
test.to_csv(r'./revtime_test.csv', index = None, header=True)
#train_file_path = 'train.csv'
#test_file_path = 'eval.csv'

In [ ]:
# Summary Stats for Train and Test dataframes
print("Train************************")
display(train.info(verbose=True))
display(train.describe(include='all').transpose().head())
display(train)

In [ ]:
print("Test************************")
display(test.info(verbose=True))
display(test.describe(include='all').transpose().head())
display(test)

In [ ]:
# plot the example time series
#!pip install seaborn
import seaborn as sns
pd.set_option('display.float_format', lambda x: '%.4f' % x)
#sns.set_context("paper", font_scale=1.3)
#sns.set_style('white')

fig, ax = plt.subplots(figsize=(24,9))
import matplotlib.pyplot as plt
whole_data = train.copy()
target_label = target_column_name
whole_data[target_label] = train[target_column_name]
for g in whole_data.groupby(time_series_id_column_names):    
    plt.plot(g[1][time_column_name].values, g[1][target_column_name].values, label=g[0])
plt.legend()
plt.show()

In [ ]:
# plot the example time series
fig, ax = plt.subplots(figsize=(24,9))
import matplotlib.pyplot as plt
whole_data = test.copy()
target_label = target_column_name
whole_data[target_label] = test[target_column_name]
for g in whole_data.groupby(time_series_id_column_names):    
    plt.plot(g[1][time_column_name].values, g[1][target_column_name].values, label=g[0])
plt.legend()
plt.show()

In [ ]:
datastore = ws.get_default_datastore()
datastore
datastore.upload_files(files = ['./revtime_train.csv', './revtime_test.csv'], target_path = 'merge_final/', overwrite = True,show_progress = True)

In [ ]:
#Create dataset for training
from azureml.core.dataset import Dataset
train_dataset = Dataset.Tabular.from_delimited_files(path=datastore.path('merge_final/revtime_train.csv'))

display(train_dataset.to_pandas_dataframe().head())
display(train_dataset.to_pandas_dataframe().tail())

In [ ]:
#Modeling

In [ ]:
#Featurization Customization examples
featurization_config = FeaturizationConfig()
featurization_config.drop_columns = ['Monthly_Return','sp500_1M_Previous']  
# Force the CPWVOL5 feature to be numeric type.
#featurization_config.add_column_purpose('CPWVOL5', 'Numeric')
# Fill missing values in the target column, Revenue, with zeros.
featurization_config.add_transformer_params('Imputer', [target_column_name], {"strategy": "constant", "fill_value": 0})
# Fill missing values in the Current Opp Period Value column with median value.
featurization_config.add_transformer_params('Imputer', ['Current_Opp._Period_Value'], {"strategy": "median"})
featurization_config.add_transformer_params('Imputer', ['Conversions'], {"strategy": "median"})
# Fill missing values in the sp500 column with forward fill (last value carried forward).
featurization_config.add_transformer_params('Imputer', ['sp500'], {"strategy": "ffill"})
featurization_config.add_transformer_params('Imputer', ['Headcount'], {"strategy": "ffill"})
featurization_config.add_transformer_params('Imputer', ['Headcount_Contingent'], {"strategy": "ffill"})

In [ ]:
# Train
from azureml.automl.core.forecasting_parameters import ForecastingParameters
lags = 'auto'#[x for x in range(1,7)]
print('lags: ',lags)
window_size = 'auto'
forecast_horizon = n_test_periods
print('window_size: ',window_size,' forecast_horizon: ',forecast_horizon)
forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    time_series_id_column_names=time_series_id_column_names,
    target_lags=lags,
    feature_lags = 'auto',
    target_rolling_window_size=window_size#,
    #drop_column_names = ['-1M Pipeline']
)

automl_config = AutoMLConfig(task='forecasting',
                             debug_log='automl_revtime_errors.log',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_hours=4,
                             training_data=train_dataset,
                             label_column_name=target_column_name,
                             compute_target=compute_target,
                             enable_early_stopping=True,
                             featurization='auto',#featurization_config,
                             n_cross_validations=5,
                             verbosity=logging.INFO,                             
                             max_concurrent_iterations=4,
                             max_cores_per_iteration=-1,
                             enable_dnn=True,
                             forecasting_parameters=forecasting_parameters)

In [ ]:
#submit a new training run
remote_run = experiment.submit(automl_config, show_output=True)
remote_run

In [ ]:
# If you need to retrieve a run that already started, use the following code
#from azureml.train.automl.run import AutoMLRun
#remote_run = AutoMLRun(experiment = experiment, run_id = 'AutoML_8b388f2d-6a16-43c3-bd53-6d134e82ce4c')

In [ ]:
remote_run.wait_for_completion()

In [ ]:
#Retrieve the Best Model
best_run, fitted_model = remote_run.get_output()
print("Best Run Model: ", best_run)
print(fitted_model.steps)
model_name = best_run.properties['model_name']
print("Model Name: ", model_name)

In [ ]:
#Transparency
#View updated featurization summary
featurization_summary = fitted_model.named_steps['timeseriestransformer'].get_featurization_summary()
# View the featurization summary as a pandas dataframe
pd.DataFrame.from_records(featurization_summary)

In [ ]:
#Explore the results
from azureml.widgets import RunDetails
RunDetails(best_run).show()

In [ ]:
#FORECASTING
X_test = test.copy()
y_test = X_test.pop(target_column_name).values

In [ ]:
X_test.tail()

In [ ]:
# forecast returns the predictions and the featurized data, aligned to X_test.
# This contains the assumptions that were made in the forecast
# The featurized data, aligned to y, will also be returned.
# This contains the assumptions that were made in the forecast
# and helps align the forecast to the original data
y_predictions, X_trans = fitted_model.forecast(X_test)

In [ ]:
#from forecasting_helper import align_outputs

df_all = align_outputs(y_predictions, X_trans, X_test, y_test)
df_all

In [ ]:
#Evaluate
#To evaluate the accuracy of the forecast, we'll compare against the actual sales quantities for some select metrics, included the mean absolute percentage error (MAPE).

#We'll add predictions and actuals into a single dataframe for convenience in calculating the metrics.

#assign_dict = {'predicted': y_predictions, target_column_name: y_test}
#df_all = X_test.assign(**assign_dict)
#df_all

In [ ]:
from azureml.automl.core.shared import constants
from azureml.automl.runtime.shared.score import scoring
from matplotlib import pyplot as plt

# use automl scoring module
scores = scoring.score_regression(
    y_test=df_all[target_column_name],
    y_pred=df_all['predicted'],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET))

print("[Test data scores]\n")
for key, value in scores.items():    
    print('{}:   {:.3f}'.format(key, value))
    
# Plot outputs
%matplotlib inline
test_pred = plt.scatter(df_all[target_column_name], df_all['predicted'], color='b')
test_test = plt.scatter(df_all[target_column_name], df_all[target_column_name], color='g')
plt.legend((test_pred, test_test), ('prediction', 'truth'), loc='upper left', fontsize=8)
plt.show()

In [ ]:
%run ./metrics_helper.ipynb

In [ ]:
df_all.groupby('horizon_origin').apply(
    lambda df: pd.Series({'MAPE': MAPE(df[target_column_name], df['predicted']),
                          'RMSE': np.sqrt(mean_squared_error(df[target_column_name], df['predicted'])),
                          'MAE': mean_absolute_error(df[target_column_name], df['predicted'])}))

In [ ]:
df_all_APE = df_all.assign(APE=APE(df_all[target_column_name], df_all['predicted']))
APEs = [df_all_APE[df_all['horizon_origin'] == h].APE.values for h in range(1, forecast_horizon + 1)]

%matplotlib inline
plt.boxplot(APEs)
plt.yscale('log')
plt.xlabel('horizon')
plt.ylabel('APE (%)')
plt.title('Absolute Percentage Errors by Forecast Horizon')

plt.show()

In [ ]:
#Confidence intervals
quantiles =  fitted_model.forecast_quantiles(X_test)
quantiles

In [ ]:
# specify which quantiles you would like 
fitted_model.quantiles = [0.01, 0.5, 0.8, 0.95]
# use forecast_quantiles function, not the forecast() one
y_pred_quantiles =  fitted_model.forecast_quantiles(X_test)

# quantile forecasts returned in a Dataframe along with the time and time series id columns 
y_pred_quantiles

In [ ]:
#Get the column index list in the right order
cols_sort_list = list(df_all.columns) + ['Revenue_Forecast']
display(cols_sort_list)
display(df_all)

In [ ]:
#Merge df_all with train
from functools import reduce
nan_value = 0
merge_dfs = [train, df_all]

final_merge_df = train.append(df_all, ignore_index=True).sort_values(by=sort_list)
coerce_df_columns_to_numeric(final_merge_df)

final_merge_df = final_merge_df.replace(np.nan, 0, regex=True)
#merge_df = merge_df.query('Fin_Entity_ID!="SGP"') # Exclude Singapore rows
#final_merge_df.sort_values(by=['Relative_Offset','Fin_Entity_ID']).reset_index(drop=True) and Relative_Month_Offset >= 0
#merge_df.drop(columns=['Current_Opp._Period_Value','Conversions'], axis=1, inplace = True)
#create new column
final_merge_df['Revenue_Forecast'] = np.where(final_merge_df['predicted']!=0, final_merge_df['predicted'], final_merge_df['Revenue'])
final_merge_df['Relative_Month_Offset'] = round((final_merge_df['End_of_Month'] - current_eom)/np.timedelta64(1,'M'),0).astype(int)
final_merge_df['End_of_Month'] = pd.to_datetime(final_merge_df['End_of_Month']).dt.date
final_merge_df['origin'] = pd.to_datetime(final_merge_df['origin']).dt.date
#inal_merge_df['Snapshot_Date_Short'] = pd.to_datetime(final_merge_df['Snapshot_Date_Short'])
final_merge_df = final_merge_df.sort_values(by=sort_list).reset_index(drop=True)
final_merge_df = final_merge_df.reindex(columns=cols_sort_list) #sort using original cols_sort_list
print("\n final_merge_df: \n")
final_merge_df.to_csv("final_merge_df.csv", index=False)
show_stats(merge_df)
final_merge_df.query('Fin_Entity_ID==@entity_debug')

In [ ]:
#pivot by EOM1, Fin_Entity_ID
final_merge_pivot1 = final_merge_df.pivot_table(index=['End_of_Month','Relative_Month_Offset','Relative_Offset'], 
                                                columns='Fin_Entity_ID', 
                                                values='Revenue_Forecast', aggfunc=np.sum, margins=True)
final_merge_pivot1 = final_merge_pivot1.reset_index(level=final_merge_pivot1.index.names)
#final_merge_pivot1['End_of_Month'] = pd.to_datetime(final_merge_pivot1['End_of_Month']).dt.date
#show_stats(revtime_pivot1)
final_merge_pivot1.tail(20)#.query('(End_of_Month == @current_eom)')

In [ ]:
# multiple line plot
#Set themes
import matplotlib.pyplot as plt
#plot
sns.set()
sns.set_context("poster") #In order of relative size they are: paper, notebook, talk, and poster. Notebook is default
sns.set(style="whitegrid") #Seaborn has five built-in themes to style its plots: darkgrid, whitegrid, dark, white, and ticks. 
# Save a palette to a variable:
palette = sns.color_palette("Accent",8)
# Use palplot and pass in the variable:
#sns.palplot(palette)
# Set the palette using the name of a palette:
#sns.set_palette(palette)

In [ ]:
#Bar plot for history and forecast revenues summary by month
fig, ax = plt.subplots(figsize=(24,9))
div_by_thousand = 1000
div_by_mil = 1000000
final_merge_df1 =final_merge_df.pivot_table(index=['End_of_Month','Relative_Month_Offset','Relative_Offset'], 
                                                #columns='Fin_Entity_ID', 
                                                values='Revenue_Forecast', aggfunc=np.sum, margins=False)
final_merge_df1 = final_merge_df1.reset_index(level=final_merge_df1.index.names)
final_merge_df1['Date'] = pd.to_datetime(final_merge_df1['End_of_Month']).dt.date#.astype(str)
final_merge_df1['Revenue_Forecast'] = round(final_merge_df1['Revenue_Forecast']/div_by_mil,4)
values = final_merge_df1['Relative_Month_Offset'].unique()
clrs = ['grey' if (x < 0) else 'g' for x in values ]
#revtime_hist21['Diff'] = round(revtime_hist21['Diff']/div_by_mil,4)
g = sns.barplot(x="Date", y="Revenue_Forecast", ci=None,
                data=final_merge_df1,
                palette=clrs, #alpha=0.50,
                ax=ax
               )
plt.xticks(rotation=90)
plt.title("Revenue & Forecast by Month (in Millions)",fontsize =18)
for p in g.patches:
    g.annotate("{:,.1f}".format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()), 
               ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')
g.legend(['Revenue','Forecast'], facecolor='w')
plt.show()

In [ ]:
div_by_thousand = 1000
div_by_mil = 1000000
final_merge_df1 = final_merge_df.copy()
final_merge_df1['Revenue_Forecast'] = round(final_merge_df1['Revenue_Forecast']/div_by_mil,4)
values = final_merge_df1['Relative_Month_Offset'].unique()
clrs = ['grey' if (x < 0) else 'g' for x in values ]
g = sns.FacetGrid(data=final_merge_df1,
                  col="Fin_Entity_ID",
                  height=8,
                  aspect=2.5,
                  col_wrap=1
                  #palette=clrs, #alpha=0.50
               )
g.map_dataframe(sns.barplot, "End_of_Month", "Revenue_Forecast", palette=clrs)
for ax in g.axes.ravel():
  for p in ax.patches:
    ax.annotate("{:,.2f}".format(p.get_height()), (p.get_x() + p.get_width() / 2., p.get_height()), 
               ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points')
g.set_xticklabels(rotation=90)
plt.title("Revenue & Forecast by Month (in Millions)",fontsize =18)
plt.show()

In [ ]:
final_merge_df.plot(x='End_of_Month', y='Revenue_Forecast')

In [ ]:
STOP*************************************************************

In [ ]:
Operationalize
#Operationalization means getting the model into the cloud so that other can run it after you close the notebook. We will create a docker running on Azure Container Instances with the model.
description = 'AutoML Revenue forecaster'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id)

In [ ]:
#Develop the scoring script
#For the deployment we need a function which will run the forecast on serialized data. It can be obtained from the best_run.
script_file_name = 'score_fcast.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

In [ ]:
#Deploy the model as a Web Service on Azure Container 
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model

inference_config = InferenceConfig(environment = best_run.get_environment(), 
                                   entry_script = script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               tags = {'type': "automl-forecasting"},
                                               description = "Automl forecasting revenue service")

aci_service_name = 'automl-revenue-forecast-01'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
aci_service.get_logs()

In [ ]:
#Call the service
import json
X_query = X_test.copy()
# We have to convert datetime to string, because Timestamps cannot be serialized to JSON.
X_query[time_column_name] = X_query[time_column_name].astype(str)
# The Service object accept the complex dictionary, which is internally converted to JSON string.
# The section 'data' contains the data frame in the form of dictionary.
test_sample = json.dumps({'data': X_query.to_dict(orient='records')})
response = aci_service.run(input_data = test_sample)
# translate from networkese to datascientese
try: 
    res_dict = json.loads(response)
    y_fcst_all = pd.DataFrame(res_dict['index'])
    y_fcst_all[time_column_name] = pd.to_datetime(y_fcst_all[time_column_name], unit = 'ms')
    y_fcst_all['forecast'] = res_dict['forecast']    
except:
    print(res_dict)

In [ ]:
y_fcst_all.head()

In [ ]:
#Delete the web service if desired
#serv = Webservice(ws, 'automl-revenue-forecast-01')
#serv.delete()     # don't do it accidentally

In [ ]:
STOP

In [ ]:
#Export revenue only for time series forecast
#cols = revhist1.columns
#revtime_hist = merge_df[cols].query('End_of_Month < @current_eom').reset_index(drop=True)
#revtime_hist = merge_final.query('End_of_Month < @current_eom').reset_index(drop=True)
#revtime_hist.to_csv("revtime_hist.csv", index=False)
#revtime_foreast = merge_final.query('End_of_Month >= @current_eom').reset_index(drop=True)
#revtime_foreast.drop(columns=['Revenue'], axis=1, inplace = True)
#revtime_foreast.to_csv("revtime_foreast.csv", index=False)

In [ ]:
#Plot dataset
sns.set()
sns.set_context("talk") #In order of relative size they are: paper, notebook, talk, and poster. Notebook is default
sns.set(style="whitegrid") #Seaborn has five built-in themes to style its plots: darkgrid, whitegrid, dark, white, and ticks. 
# Save a palette to a variable:
palette = sns.color_palette("Accent",8)
# Use palplot and pass in the variable:
sns.palplot(palette)
# Set the palette using the name of a palette:
sns.set_palette(palette)

dataset = revtime_hist
dataset['End_of_Month'] = pd.to_datetime(dataset['End_of_Month'])
dataset = dataset.set_index('End_of_Month')
values = dataset.values
# specify columns to plot
groups = [0,1] #, 2, 3, 5, 6, 7]
i = 1
# plot each column
plt.figure()
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(values[:, group])
    plt.title(dataset.columns[group], y=0.1, loc='right')
    i += 1
plt.show()

In [ ]:
dataset['Revenue'].plot()